In [1]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import os
import json
import numpy as np
import time
import random
import json
from concurrent.futures import ThreadPoolExecutor, as_completed
from datetime import datetime, timedelta
from tika import parser
from bs4.element import Comment
from bs4 import BeautifulSoup
from glob import glob

In [2]:
TIKA_HOST = 'http://localhost:9998'

def tag_visible(element):
    if element.parent.name in ['style', 'script', 'head', 'title', 'meta', '[document]']:
        return False
    if isinstance(element, Comment):
        return False
    return True


def text_from_html(body, is_html=True):
    if is_html:
        soup = BeautifulSoup(body, 'html.parser')
    else:
        soup = body
    texts = soup.findAll(text=True)
    visible_texts = filter(tag_visible, texts)
    return '\n'.join(t.strip() for t in visible_texts).strip()


def parse_tika(file, minlen=2):
    raw_xml = parser.from_file(file, TIKA_HOST, xmlContent=True)
    body = BeautifulSoup(raw_xml['content']).find('body')
    t = []
    for b in body:
        try:
            t_ = text_from_html(b, is_html=False)
            if len(t_) > minlen:
                t.append(t_)
        except Exception as e:
            pass
    
    return t

In [3]:
files = glob('pdf/*')
len(files)

25766

In [4]:
with open('all_a.json') as fopen:
    all_a = json.load(fopen)

In [5]:
all_a[3775]

'https://medic.upm.edu.my/upload/dokumen/2020042010180819_MJMHS_0508.pdf'

In [6]:
with open('edu.my.jsonl', 'w') as fopen:
    for f in tqdm(files):
        try:
            t = '\n'.join(parse_tika(f))
            index = int(f.split('/')[1].split('.')[0])
            data = {
                'file': f,
                'text': t,
                'url': all_a[index]
            }
            fopen.write(f'{json.dumps(data)}\n')
        except Exception as e:
            print(e)

 40%|██████████████▋                      | 10214/25766 [11:38<14:50, 17.46it/s]

object of type 'NoneType' has no len()


 67%|████████████████████████▋            | 17184/25766 [19:02<05:02, 28.39it/s]

object of type 'NoneType' has no len()


100%|█████████████████████████████████████| 25766/25766 [28:21<00:00, 15.14it/s]
